In [19]:
###IMPORTANT ####
 # Actor until 1976
 # Actor in a Leading Role since 1977
 # Same for Actress
    
### Stock the data better than doing a request every try # Done

### PROBLEME : COMMENT GERER LES 2 WINNERS, existe plusieurs fois # Done

### PROBLEM : for DIRECTION, names and titles are swapped # Done


### PROBLEME : Incohérence des noms de catégories sur plusieurs année : best motion picture -> best picture
# Special effect -> Visual effects etc

### Les informations récoltées ne sont pas les mêmes ex : foreign film c'est le pays qui est récolté, comment faire ?

# * bonjour
# ! alert 

import sys
!{sys.executable} -m pip install jellyfish

In [4]:
test.getFILMS()

1
0
Getting the content of oscars.org webpages ...


100%|██████████| 21/21 [00:01<00:00, 14.58it/s]


Extracting the data of the website ...


100%|██████████| 21/21 [00:00<00:00, 24.84it/s]


Requesting TheMovieDataBase API for the gender and birthdays ...


100%|██████████| 205/205 [00:00<00:00, 413380.92it/s]

                               film                 name   result  gender  \
year category                                                               
1930 Directing      The Divine Lady          Frank Lloyd   Winner    Male   
     Directing             Madame X     Lionel Barrymore  Nominee    Male   
     Directing  The Broadway Melody       Harry Beaumont  Nominee    Male   
     Directing       In Old Arizona      Irving Cummings  Nominee    Male   
     Directing                 Drag          Frank Lloyd  Nominee    Male   
...                             ...                  ...      ...     ...   
1950 Actress            The Heiress  Olivia de Havilland   Winner  Female   
     Actress                  Pinky         Jeanne Crain  Nominee  Female   
     Actress       My Foolish Heart        Susan Hayward  Nominee  Female   
     Actress         Edward, My Son         Deborah Kerr  Nominee  Female   
     Actress     Come to the Stable        Loretta Young  Nominee  Female   